# Airplane allocation - A stochastic problem

## Problem definition

In a world where everything move fast, airline companies have to adapt their airplane allocation in order to fit as best as they can the passanger demand. To reach that goal, one of the possible method is using a stochastic approach. In fact, many variables must be digest to find one the best possible airplane distribution. A stochastic approach is well fitted for our case because it's allow to search one possible solution among a large set of possible one by trying a lot of airplanes distribution solutions.

In our proble, an airline company wishes to allocate airplanes of various types among its route to satisfy an uncertain passenger demand, in such a way as to minimize operating costs plus the lost revenue from passengers turned away.
This problem use *4* different types of aircraft and *5* routes.

This problem is taken from the book *Stochastic Programming Problems: Examples from the Literature* by Alan King. In his book, Mr. King introduces use one the possbile solutionn with his own set of arguments : 

In [1]:
from IPython.display import IFrame
plot_fn = 'assets/files/airlanes.pdf'
IFrame(plot_fn, width=1000, height=500)

In the following report, we will introduce us our work and our implementation of this airplane allocation problem

## The modeling
In order to modelize the problem, we use a matrix $A$ containing the number of aircraft use for a given route and a given type :

$A = (x_{i,j})_{1\leq i\leq 5, 1\leq j\leq 4} = \left( \begin{array}{cccc}
x_{1,1} & \ldots & \ldots & x_{1,4} \\
x_{2,1} & \ldots & \ldots & x_{2,4} \\
x_{3,1} & \ldots & \ldots & x_{3,4} \\
x_{4,1} & \ldots & \ldots & x_{4,4} \\ 
x_{5,1} & \ldots & \ldots & x_{5,4} \\ \end{array} \right) \in M_{5,4}(\mathbb{N})$ where $x \in \mathbb{N}$

Where $x_{i,j}$ represents the number of airplane for the type $j$ on the route $i$.

Example $x_{3,4} = 23$ means that we have 23 aircrafts of type 4 on the route number 3

## The cost function

In order to find the best solution, we need to minimize the following cost function :

<br>
<center>$f = \sum_{i=1}^{5} \sum_{j=1}^{4}( Cost_{i,j} * x_{i,j})+ \sum_{k=1}^{5} RevenueLost_{k} * PassengersTA_{k}$</center>

Where :

* $Cost_{i,j}$ characterize the operational cost of an aircraft by the route.
For our implementation, we chose to use this matrix $C \in $ of operational costs :

$C = \left( \begin{array}{cccc}
12 & 20 & 20 & 19 \\
2 & 34 & 10 & 20 \\
43 & 63 & 40 & 12 \\
32 & 10 & 6 & 34 \\ 
20 & 30 & 10 & 87 \\ \end{array} \right)\in M_{5,4}(\mathbb{N})$

Example : $Cost_{1,4} = 19$ and $Cost_{5,2} = 30$

* $RevenueLost_{k}$ with $k \in [1,5]$ denote the revenue lost per passenger turned away on the $kth$ route.
For our implementation, we chose to use this matrix $R$ :

$R = \left( \begin{array}{cccc}
13 & 20 & 7 & 7 & 15\\ \end{array} \right)\in M_{1,5}(\mathbb{N})$

* $PassengersTA_{k} \in \mathbb{N} = D_{1,k} - \sum_{j=1}^{4}(A_{k,j}*B_{k,j})$ with $k \in [1,5]$ denote the number of passenger who turned away from the $kth$ route. This is obtained by substracting the route demand by the real number of passenger knew by multiplying the number of aircraft per route by their capacity. The demand is denote by this matrix $D$:

$D = \left( \begin{array}{cccc}
800 & 900 & 700 & 650 & 380\\ \end{array} \right)\in M_{1,5}(\mathbb{N})$

And the total capacity of each type of aircraft is denoted by the following matrix $B$ : 

$B = \left( \begin{array}{cccc}
16 & 10 & 30 & 23 \\ \end{array} \right)\in M_{1,4}(\mathbb{N})$



**Our goal here will to lower this function in order to fit the demand with the lowest possible cost. To achieve this goal, we choose two approach. The first one is the resoltion by a population based algorithm, the genetic one.**

## The proposed solution - Population based algorithm
The main idea behind this type of algorithm is to generate multiple generation and for each one pick the two best members, cross them, mutate the resulted child and generate a new popuation :

### Key features

* **Population size** : We chose to use a single population. The size is kept constant across all generations
* **Mixing operations** : To generate our children, we use crossover among the the parent matrix column. When our children are generated, they are then mutated by re-generating the entire given column
* **Succession scheme** : We use the *µ,µ* succession scheme. This means that with *µ* parents we get *µ* children in our new generation
* **Stopping condition** : As we are looking for the one of the best solution, our stopping condition is the generation length. 

### The algorithm

1. Generate a base population $P = \{p_0,...,p_n\}$ of size $n$. The first iteration will generate a random one with one constraint, we have a limited number $T_{1,k}$ of each type $k$ of aircraft : $T = \left( \begin{array}{cccc}
10 & 19 & 25 & 16\\ \end{array} \right)\in M_{1,5}(\mathbb{N})$
2. Compute the total cost of the generation with the our formula previously define.
4. Compute the probality for each individuals to picked them regarding their cost : $P \in [0,1] = \begin{eqnarray}\frac{1/Cost}{\sum_{}1/Cost} \end{eqnarray}$
3. Pick the to best members $\{p_1,p_2\}$ of the population according to their probability. They will be our two *parents*.
4. Cross both parents according to a linear probality $v \in [0,1]$  and a threshold $C_T \in [0,1]$. For each column of the parents, if $v \leq C_T$ we swap the colum. At the end we have a new child. This is done two times to get two children
5. Mutate both children according to a linear probality $v \in [0,1]$ and a threshold $M_T \in [0,1]$. For each column of the children, if $v \leq M_T$ we swipe two random value in the same row
6. One of the two child is picked and added to the new generation.
7. We repeat *4, 5 and 6* until the size of the new generation is equal to the previous one ($n$).

### Set of research

Let's denote $(\Omega, \mathcal{F}, \mathcal{P})$ our probalistic space : 

<center>$\Omega=\{M_{1},...,M_{n}\}$ with $M \in M_{5,4}(\mathbb{N})$</center>
<br>
<center>$\mathcal{F}=\{\omega_{1},...,\omega_{n}\}$ with $\omega \in \Omega$ and $n=population\ size$</center>
<br>
<center>$P \in [0,1] = \begin{eqnarray}\frac{1/Cost}{\sum_{}1/Cost} \end{eqnarray}\ (cf. Cost\ function\ paragraphe)$<center>


// DESCRIBE THE SET OU ON CHERCHE
// REFER TO THE FIRST SECTION
// REFER TO THE COST FUNCTION SECTION





// EN DESSOUS
// ENUMERATE THE GOAL OF THESE TESTS. USE MUTATION SO REACH ANY SOLUTION FROM THE ADMISSIBLE SET WITH PROBA != 0 = CONVERGENCE
// SHOW NO LOCAL MINIMIZER
// THE GOAL OF BENCHMARK TEST (MIN COMPUTE COST OR OTHERS)
// DESCRIBE THE PLAN OF TESTS (GENEARATION, POP SIZE, CHANGING OPERATION) ==> INCLUDE SOME REMARKS
// PRESENT IN THE BEST ORDER THE RESULTS OF TEST

## First implementation - Probabilistic pickup

<img src="assets/images/1_airplanes_g400_i30_cT0.3_mT0.15.png">

This graph shows you for every generation, the best cost we can get. In the run, we run *400* generations of *30* individuals.
Unfortunately, the result is not converging toward a possible solution but it is chaotic. This is because of the way we pick our parents. We try in a first place to  If we base this on the probabilities, the results are not good. But if pick the two best of the population, the results are much better.

In this first run, $C_{T} = 0.3$ and $M_{T}=0.15$

## Second implementation - An agressive pick up

<img src="assets/images/2_airplanes_g400_i30_cT0.3_mT0.15.png">

For this second implementation, we decided to change the way of picking the parents responsible to generate the future generation. Instead of picking them according to there probability, we directly pick the two best element of the population.
So we change the fith step of the algorithm by this : 
5. Pick the to best members $\{p_1,p_2\}$ of the population according to their cost. The parents with the lowest cost among all the population. They will be our two *parents*: $p_1 = \underset{x \in P}{min} f(x) \quad \textrm{and} \quad p_2 = \underset{x \in P-{p_1}}{min} f(x)$.

As you can see we have a covergence toward one of the possible solution and no more chaos.

In this second run,  $C_{T} = 0.3$ and $M_{T}=0.15$


# CONCLUSION

// GENERAL CONCLUSION (SUCCESSFULL OR NOT AND WHY)
// MORE DETAIL (SELECT MORE MUTATION OR MORE CROSSING ...)
// OPENING, SHOW FUTURE RESEARCH